In [1]:
import os

from model import get_personlab
from tf_data_generator import *
from config import config
from keras.models import load_model
from keras.utils import multi_gpu_model
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback
from keras import backend as KB

from polyak_callback import PolyakMovingAverage

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

Using TensorFlow backend.


In [2]:

# Only allocate memory on GPUs as used
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth=True
set_session(tf.Session(config=tf_config))

LOAD_MODEL_FILE = config.LOAD_MODEL_PATH
SAVE_MODEL_FILE = config.SAVE_MODEL_PATH
H5_DATASET = config.H5_DATASET

print(LOAD_MODEL_FILE, SAVE_MODEL_FILE, H5_DATASET)

num_gpus = config.NUM_GPUS
batch_size_per_gpu = config.BATCH_SIZE_PER_GPU
if num_gpus > 1:
    batch_size = num_gpus * batch_size_per_gpu    
else:
    batch_size = 1

None ./personlab_model.h5 coco2017_personlab_train.h5


In [3]:

input_tensors = get_data_input_tensor(batch_size=batch_size)
for i in range(len(input_tensors)):
    input_tensors[i].set_shape((None,)+input_shapes[i])

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [4]:
if num_gpus > 1:
    with tf.device('/cpu:0'):
        model = get_personlab(train=True, input_tensors=input_tensors, with_preprocess_lambda=True)
else:
    model = get_personlab(train=True, input_tensors=input_tensors, with_preprocess_lambda=True)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
if LOAD_MODEL_FILE is not None:
    model.load_weights(LOAD_MODEL_FILE, by_name=True)

In [6]:
if num_gpus > 1:
    parallel_model = multi_gpu_model(model, num_gpus)
else:
    parallel_model = model

In [7]:
for loss in parallel_model.outputs:
    parallel_model.add_loss(loss)

In [8]:
def save_model(epoch, logs):
    model.save_weights(SAVE_MODEL_FILE)

In [9]:

# In Keras, this metric will not be computed for this model, since the outputs have no targets.
# Only by commenting out that restriction in the Keras code will allow the display of these metrics
# which can be used to monitor the individual losses.
def identity_metric(y_true, y_pred):
    return KB.mean(y_pred)


callbacks = [LambdaCallback(on_epoch_end=save_model)]

if config.POLYAK:
    def build_save_model():
        with tf.device('/cpu:0'):
            save_model = get_personlab(train=True, input_tensors=input_tensors, with_preprocess_lambda=True)
        return save_model
    polyak_save_path = '/'.join(config.SAVE_MODEL_FILE.split('/')[:-1]+['polyak_'+config.SAVE_MODEL_FILE.split('/')[-1]])
    polyak = PolyakMovingAverage(filepath=polyak_save_path, verbose=1, save_weights_only=True,
                                    build_model_func=build_save_model, parallel_model=True)

    callbacks.append(polyak)

In [10]:

# The paper uses SGD optimizer with lr=0.0001
parallel_model.compile(target_tensors=None, loss=None, optimizer=Adam(), metrics=[identity_metric])


In [11]:
parallel_model.fit(steps_per_epoch=64115//batch_size, 
                   epochs=config.NUM_EPOCHS, 
                   callbacks=callbacks)


Instructions for updating:
Use tf.cast instead.
Epoch 1/125


InvalidArgumentError: TypeError: 'KeysViewHDF5' object is not subscriptable
Traceback (most recent call last):

  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/ops/script_ops.py", line 207, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 449, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/ghafran/git/KerasPersonLab/tf_data_generator.py", line 89, in data_gen
    random.shuffle(keys)

  File "/usr/local/Cellar/python/3.7.3/Frameworks/Python.framework/Versions/3.7/lib/python3.7/random.py", line 278, in shuffle
    x[i], x[j] = x[j], x[i]

TypeError: 'KeysViewHDF5' object is not subscriptable


	 [[{{node PyFunc}}]]
	 [[{{node IteratorGetNext}}]]